https://github.com/pytorch/examples/blob/0.3/mnist_hogwild/main.py
https://pytorch.org/docs/stable/notes/multiprocessing.html

### Load Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

### Load Data

In [2]:
wine_data = pd.read_csv('data/wine_data.csv')

In [3]:
wine_features = wine_data.drop('Class', axis = 1)
wine_target = wine_data[['Class']]

In [4]:
X_train, x_test, Y_train, y_test = train_test_split(wine_features,
                                                    wine_target,
                                                    test_size=0.4,
                                                    random_state=0)

In [5]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(x_test.values).float()
Ytrain_ = torch.from_numpy(Y_train.values).view(1,-1)[0]
Ytest_ = torch.from_numpy(y_test.values).view(1,-1)[0]

### Define Model

In [8]:
input_size = 13
output_size = 3
hidden_size = 100

In [9]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        X = torch.sigmoid((self.fc1(X)))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)

        return F.log_softmax(X, dim=-1)

### Define train method

In [6]:
def train(model, Xtrain_, Ytrain_,  optimizer):
    
    for epoch in range(1, 1001):
        
        model.train()
        pid = os.getpid()

        optimizer.zero_grad()
        Ypred = model(Xtrain_)

        loss = nn.NLLLoss()(Ypred , Ytrain_)
        loss.backward()

        optimizer.step()

        if epoch % 100 == 0:
                print('{}\tTrain Epoch: {} \tLoss: {:.6f}'.format(
                    pid, epoch, loss.item()))
                

## Training the model
* torch.multiprocessing is a drop in replacement for Python’s multiprocessing module. 
* All tensors sent through a multiprocessing.Queue, will have their data moved into shared memory and will only send a handle to another process.
* We first train the model across `num_processes` processes
* subprocesses are created by calling set_start_method
* model.share_memory() shares the model across the processes
* rank refers to process hierarchy


In [10]:
import torch.multiprocessing as mp

mp.get_all_sharing_strategies()

{'file_system'}

In [11]:
mp.get_sharing_strategy()

'file_system'

In [12]:
if __name__ == '__main__':
    
    mp.set_start_method('fork')
 
    Xtrain_.share_memory_()
    Ytrain_.share_memory_() 
    Xtest_.share_memory_()
    Ytest_.share_memory_()

    model = Net()
    model.share_memory() # gradients are allocated lazily, so they are not shared here
    
    optimizer = optim.Adam(model.parameters(), lr = 0.001)

    processes = []
    for rank in range(4):
        
        p = mp.Process(target=train, args=(model, Xtrain_, Ytrain_, optimizer ))
        p.start()
        processes.append(p)
        
    for p in processes:
        p.join()
        
    if(processes[0].is_alive()==False):
        
        model.eval()
        test_loss = 0
        correct = 0

        predict_out = model(Xtest_)
        _, predict_y = torch.max(predict_out, 1)
        
        print("\n")
        print ('prediction accuracy', accuracy_score(Ytest_.data, predict_y.data))
        print ('micro precision', precision_score(Ytest_.data, predict_y.data, average='micro'))
        print ('micro recall', recall_score(Ytest_.data, predict_y.data, average='micro'))

90450	Train Epoch: 100 	Loss: 0.640276
90452	Train Epoch: 100 	Loss: 0.632746
90451	Train Epoch: 100 	Loss: 0.611317
90453	Train Epoch: 100 	Loss: 0.599894
90450	Train Epoch: 200 	Loss: 0.516419
90452	Train Epoch: 200 	Loss: 0.510591
90451	Train Epoch: 200 	Loss: 0.484444
90453	Train Epoch: 200 	Loss: 0.434364
90452	Train Epoch: 300 	Loss: 0.535437
90450	Train Epoch: 300 	Loss: 0.514480
90451	Train Epoch: 300 	Loss: 0.431045
90453	Train Epoch: 300 	Loss: 0.322277
90451	Train Epoch: 400 	Loss: 0.226990
90450	Train Epoch: 400 	Loss: 0.139121
90452	Train Epoch: 400 	Loss: 0.145423
90453	Train Epoch: 400 	Loss: 0.193741
90452	Train Epoch: 500 	Loss: 0.135082
90450	Train Epoch: 500 	Loss: 0.113474
90451	Train Epoch: 500 	Loss: 0.127406
90453	Train Epoch: 500 	Loss: 0.121229
90451	Train Epoch: 600 	Loss: 0.107233
90452	Train Epoch: 600 	Loss: 0.127822
90450	Train Epoch: 600 	Loss: 0.129779
90453	Train Epoch: 600 	Loss: 0.082163
90450	Train Epoch: 700 	Loss: 0.149483
90452	Train Epoch: 700 	L